In [1]:
import requests
import zipfile
import io
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
"https://app.enappsys.com/datadownload?code=hu/elec/pricing/daprices&currency=EUR&delimiter=comma&minavmax=false&pass=204245237229238244242225228229178224178181&res=qh&tag=csv&timezone=CET&user=peter.koleszar@solservices.hu&start=202501130000&end=202510190000"

'https://app.enappsys.com/datadownload?code=hu/elec/pricing/daprices&currency=EUR&delimiter=comma&minavmax=false&pass=204245237229238244242225228229178224178181&res=qh&tag=csv&timezone=CET&user=peter.koleszar@solservices.hu&start=202501130000&end=202510190000'

In [3]:
import io
from datetime import datetime, timedelta

# Define one week from each season in 2024
# January (winter), April (spring), June (summer), October (fall)
weeks_2024 = [
    ("202401080000", "202401150000"),  # January week (Jan 8-15)
    ("202404080000", "202404150000"),  # April week (Apr 8-15)
    ("202406100000", "202406170000"),  # June week (Jun 10-17)
    ("202410070000", "202410140000"),  # October week (Oct 7-14)
]

API_KEY = "204245237229238244242225228229178176178181"
BASE_URL = "https://app.enappsys.com/datadownload"

all_dfs = []
for i, (start, end) in enumerate(weeks_2024, 1):
    url = (
        f"{BASE_URL}?code=hu/elec/pricing/daprices"
        f"&currency=EUR"
        f"&delimiter=comma"
        f"&minavmax=false"
        f"&pass={API_KEY}"
        f"&res=hourly"
        f"&tag=csv"
        f"&timezone=CET"
        f"&user=peter.koleszar@solservices.hu"
        f"&start={start}"
        f"&end={end}"
    )
    
    resp = requests.get(url, timeout=30)
    resp.raise_for_status()
    
    df_week = pd.read_csv(io.StringIO(resp.text))
    all_dfs.append(df_week)
    print(f"Week {i}/4: {start[:8]} to {end[:8]} -> {len(df_week)} rows")

# Concatenate all weeks
montel_df = pd.concat(all_dfs, ignore_index=True)
print(f"\nTotal downloaded: {len(montel_df)} rows from 4 weeks across 2024 seasons")


Week 1/4: 20240108 to 20240115 -> 169 rows
Week 2/4: 20240408 to 20240415 -> 169 rows
Week 3/4: 20240610 to 20240617 -> 169 rows
Week 4/4: 20241007 to 20241014 -> 169 rows

Total downloaded: 676 rows from 4 weeks across 2024 seasons


In [4]:
montel_df

,Date (CET),AUSTRIA (AT),CROATIA (HR),GERMANY (DE),HUNGARY (HU),ROMANIA (RO),SERBIA (RS),SLOVAKIA (SK),SLOVENIA (SL)
0,NaN,EUR/MWh,EUR/MWh,EUR/MWh,EUR/MWh,EUR/MWh,EUR/MWh,EUR/MWh,EUR/MWh
1,[08/01/2024 00:00],82.07,82.45,87.02,83.15,78.06,80.08,83.15,83.15
2,[08/01/2024 01:00],81.09,81.10,84.32,82.73,74.36,76.14,82.75,81.94
3,[08/01/2024 02:00],72.68,72.67,81.82,77.47,51.69,74.00,77.50,75.16
4,[08/01/2024 03:00],69.96,68.73,79.11,75.22,40.56,70.09,75.05,72.04
...,...,...,...,...,...,...,...,...,...
671,[13/10/2024 19:00],74.86,101.41,54.05,121.77,143.00,130.08,41.90,87.84
672,[13/10/2024 20:00],63.27,84.74,39.51,100.22,120.00,104.12,34.87,73.75
673,[13/10/2024 21:00],49.28,65.08,38.43,75.67,90.45,81.19,25.33,56.86
674,[13/10/2024 22:00],46.32,52.42,40.13,56.03,58.55,64.26,43.02,49.88


In [5]:
montel_df = montel_df[["HUNGARY (HU)", "Date (CET)"]]

In [6]:
montel_df

,HUNGARY (HU),Date (CET)
0,EUR/MWh,NaN
1,83.15,[08/01/2024 00:00]
2,82.73,[08/01/2024 01:00]
3,77.47,[08/01/2024 02:00]
4,75.22,[08/01/2024 03:00]
...,...,...
671,121.77,[13/10/2024 19:00]
672,100.22,[13/10/2024 20:00]
673,75.67,[13/10/2024 21:00]
674,56.03,[13/10/2024 22:00]


In [7]:
# Inspect the date format first
print("Sample date values:")
print(montel_df["Date (CET)"].head())
print(f"\nFirst date type: {type(montel_df['Date (CET)'].iloc[0])}")
print(f"First date value: {repr(montel_df['Date (CET)'].iloc[0])}")

# Convert date column to datetime (strip brackets if present)
montel_df["Date (CET)"] = montel_df["Date (CET)"].astype(str).str.strip("[]")
montel_df["Date (CET)"] = pd.to_datetime(montel_df["Date (CET)"], format="%d/%m/%Y %H:%M", errors="coerce")

print(f"\nConverted date column to datetime. Date range: {montel_df['Date (CET)'].min()} to {montel_df['Date (CET)'].max()}")
print(f"Failed conversions: {montel_df['Date (CET)'].isna().sum()}")


Sample date values:
0                   NaN
1    [08/01/2024 00:00]
2    [08/01/2024 01:00]
3    [08/01/2024 02:00]
4    [08/01/2024 03:00]
Name: Date (CET), dtype: object

First date type: <class 'float'>
First date value: nan

Converted date column to datetime. Date range: 2024-01-08 00:00:00 to 2024-10-13 23:00:00
Failed conversions: 4


C:\Users\Baris Sever\AppData\Local\Temp\ipykernel_19836\4248325296.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  montel_df["Date (CET)"] = montel_df["Date (CET)"].astype(str).str.strip("[]")
C:\Users\Baris Sever\AppData\Local\Temp\ipykernel_19836\4248325296.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  montel_df["Date (CET)"] = pd.to_datetime(montel_df["Date (CET)"], format="%d/%m/%Y %H:%M", errors="coerce")


In [8]:
# Convert price column to float and set date as index
montel_df["HUNGARY (HU)"] = pd.to_numeric(montel_df["HUNGARY (HU)"], errors="coerce")
montel_df = montel_df.set_index("Date (CET)", drop=True)

print(f"Price column converted to float. Data shape: {montel_df.shape}")
print(f"Index is datetime: {isinstance(montel_df.index, pd.DatetimeIndex)}")
print(f"\nSample data:")
print(montel_df.head())


Price column converted to float. Data shape: (676, 1)
Index is datetime: True

Sample data:
                     HUNGARY (HU)
Date (CET)                       
NaT                           NaN
2024-01-08 00:00:00         83.15
2024-01-08 01:00:00         82.73
2024-01-08 02:00:00         77.47
2024-01-08 03:00:00         75.22


C:\Users\Baris Sever\AppData\Local\Temp\ipykernel_19836\4181801999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  montel_df["HUNGARY (HU)"] = pd.to_numeric(montel_df["HUNGARY (HU)"], errors="coerce")


In [ ]:
# Drop all rows where price is NaN
initial_count = len(montel_df)
nan_count = montel_df["HUNGARY (HU)"].isna().sum()
montel_df = montel_df.dropna(subset=["HUNGARY (HU)"])
final_count = len(montel_df)

print(f"Dropped {initial_count - final_count} rows with NaN prices")
print(f"Remaining rows: {final_count}")
print(f"NaN values remaining: {montel_df['HUNGARY (HU)'].isna().sum()}")


Dropped 4 rows with NaN prices
Remaining rows: 672
NaN values remaining: 0


: 